## Вариант 1

**Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:**

- Наименование вакансии.
- Предлагаемую зарплату (отдельно минимальную и максимальную).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия. 

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). 
Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.


In [1]:
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
def search_hh(vacancy, max_pages):
    vac_list = []
    page_counter = 0

    params = {
        'text': vacancy,
        'search_field': 'name',
        'page': 0
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'
    }

    url = 'https://hh.ru/search/vacancy'

    while page_counter <= max_pages:
        response = requests.get(url, params=params, headers=headers)

        if not response.ok:
            return vac_list

        dom = bs(response.text, 'html.parser')
        vacancy_divs = dom.find_all(
            'div', {'class': 'vacancy-serp-item'})

        for vac_div in vacancy_divs:
            vac_data = {}
            vac_data['site'] = 'hh.ru'
            vac_data_a = vac_div.find(
                'a', {'data-qa': 'vacancy-serp__vacancy-title'})

            vac_data['name'] = None
            
            if vac_data_a:
                vac_data['name'] = vac_data_a.getText()

                vac_data_url = vac_data_a['href']
                vac_data['url'] = urljoin(
                    vac_data_url, urlparse(vac_data_url).path)

            if vac_data['name'] is None:
                continue
                
            vac_data_employer = vac_div.find(
                'a', {'data-qa': 'vacancy-serp__vacancy-employer'})
            if vac_data_employer:
                vac_data['employer'] = vac_data_employer.getText()

            vac_data_location = vac_div.find(
                'span', {'data-qa': 'vacancy-serp__vacancy-address'})
            if vac_data_location:
                vac_data['location'] = vac_data_location.getText()

            vac_data_salary = vac_div.find(
                'span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
            if vac_data_salary:
                salary = vac_data_salary.getText()
                salary_list = salary.split('–')

                if len(salary_list) == 1:
                    vac_data['salary'] = ''.join([str(s)
                                                  for s in salary_list[0].split() if s.isdigit()])

                elif len(salary_list) == 2:
                    vac_data['salary_from'] = ''.join([str(s)
                                                       for s in salary_list[0].split() if s.isdigit()])

                    vac_data['salary_to'] = ''.join([str(s)
                                                     for s in salary_list[1].split() if s.isdigit()])


            vac_list.append(vac_data)

        page_counter += 1
        if page_counter <= max_pages:
            next_page_a = dom.find('a', {'data-qa': 'pager-next'})

            if next_page_a:
                params['page'] = page_counter
            else:
                break

    return vac_list

In [4]:
def search_superjob(vacancy, max_pages):
    vac_list = []
    page_counter = 1

    params = {
        'keywords': vacancy,
        'profession_only': 1,
        'page': 1
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'
    }

    url = 'https://superjob.ru/vacancy/search/'

    while page_counter <= max_pages:
        response = requests.get(url, params=params, headers=headers)

        if not response.ok:
            return vac_list

        dom = bs(response.text, 'html.parser')
        vacancy_divs = dom.find_all(
            'div', {'class': 'f-test-search-result-item'})

        for vac_div in vacancy_divs:
            vac_data = {}
            vac_data['site'] = 'superjob.ru'

            vac_name = vac_div.find(
                'div', {'class': 'jNMYr GPKTZ _1tH7S'})

            vac_data['name'] = None
            if vac_name:
                vac_a = vac_name.find('a')
                if vac_a:
                    if vac_a['href']:
                        vac_data['url'] = 'https://superjob.ru' + vac_a['href']
                        
                    vac_data['name'] = vac_a.getText()

            if vac_data['name'] is None:
                continue
                    
            vac_employer = vac_div.find(
                'span', {'class': 'f-test-text-vacancy-item-company-name'})

            if vac_employer:
                vac_vac_employer_a = vac_employer.find('a')

                if vac_vac_employer_a:
                    vac_data['employer'] = vac_vac_employer_a.getText()

            vac_loc = vac_div.find(
                'span', {'class': 'f-test-text-company-item-location'})

            if vac_loc:
                vac_loc_list = vac_loc.findChildren()
                if len(vac_loc_list) == 3:
                    vac_data['location'] = vac_loc_list[2].getText()

            vac_data_salary = vac_div.find(
                'span', {'class': 'f-test-text-company-item-salary'})
            if vac_data_salary:
                salary = vac_data_salary.getText()
                salary_list = salary.split('—')

                if len(salary_list) == 1:
                    vac_data['salary'] = ''.join([str(s)
                                                  for s in salary_list[0].split() if s.isdigit()])

                elif len(salary_list) == 2:
                    vac_data['salary_from'] = ''.join([str(s)
                                                       for s in salary_list[0].split() if s.isdigit()])

                    vac_data['salary_to'] = ''.join([str(s)
                                                     for s in salary_list[1].split() if s.isdigit()])

            vac_list.append(vac_data)

        page_counter += 1
        if page_counter <= max_pages:
            next_page_a = dom.find(
                'a', {'class': 'f-test-button-keyboard_arrow_right'})

            if next_page_a:
                params['page'] = page_counter
            else:
                break

    return vac_list

In [5]:
def search_vacancies(vacancy, max_pages):

    vac_data = []
    vac_data.extend(search_hh(vacancy, max_pages))
    vac_data.extend(search_superjob(vacancy, max_pages))
    df = pd.DataFrame(vac_data)

    return df

In [6]:
vac_str = input('Подстрока в наименовании вакансии: ')
max_pages = int(input('Максимальное количество страниц с вакансиями: '))
print(f'Показаны вакансии с сайтов hh.ru и superjob.ru. Подстрока поиска: {vac_str}. Макс. количество страниц с вакансиями: {max_pages}.')

Подстрока в наименовании вакансии:  QT
Максимальное количество страниц с вакансиями:  2


Показаны вакансии с сайтов hh.ru и superjob.ru. Подстрока поиска: QT. Макс. количество страниц с вакансиями: 2.


In [7]:
search_vacancies(vac_str, max_pages)

,site,name,url,employer,location,salary_from,salary_to,salary
0,hh.ru,"Программист C++, Qt",https://hh.ru/vacancy/47793265,ООО АВАНТИС3Д,"Москва, Калужская",170000,220000,NaN
1,hh.ru,"Программист C++ Qt | ""ПОЛАТОР""",https://hh.ru/vacancy/47854066,ООО Инженерная компания,"Москва, Калужская",200000,300000,NaN
2,hh.ru,Программист C++/Qt/QML,https://hh.ru/vacancy/47676538,ООО ДЕМЛАБС,Новосибирск,90000,150000,NaN
3,hh.ru,Программист С++/Qt,https://hh.ru/vacancy/47752344,ЗАО НПЦ ИРС,"Москва, Калужская",50000,95000,NaN
4,hh.ru,Разработчик интерфейсов (Программист на QT/PyQT),https://hh.ru/vacancy/46889639,ФГУП ГосНИИАС,Москва,90000,150000,NaN
5,hh.ru,Программист C++/Qt (middle),https://hh.ru/vacancy/47239653,Компания ДЭП,"Москва, Пражская",120000,140000,NaN
6,hh.ru,QT программист - разработчик,https://hh.ru/vacancy/47716059,ООО ВЕСТА,Санкт-Петербург,NaN,NaN,NaN
7,hh.ru,Разработчик Qt/QML,https://hh.ru/vacancy/47583770,ООО КВАНТ ПРОГРАММ,"Москва, Перово",NaN,NaN,NaN
8,hh.ru,Программист С++/Qt (middle),https://hh.ru/vacancy/47562829,ООО Аргун-Софт,"Санкт-Петербург, Московские ворота",NaN,NaN,80000
9,hh.ru,Разработчик С++/Qt,https://hh.ru/vacancy/46537539,ЗАО Топ Системы,"Москва, Дмитровская",NaN,NaN,NaN
